# Project - 1 : Apache Spark 

Instructor: Ramesh Yerraballi
TA: Madhumitha Sakthi
Semester: Fall 2019
Due Date: 11:59pm, Monday 9/16

This project is based on Map-Reduce Framework. In these you will get to work with Spark and will get to know how 
does spark work, what functionalities does spark provide, what does map-reduce framework do and why is it useful. 

In this project you will be implementing a basic song recommender system. You will be given a dataset where there are multiple csv files. These csv files have data corresponding to song play count and song information.

The data you would be using will be provided in a zip file along with this notebook. The __msd.zip__ archive contains '_kaggle_visible_evaluation_triplets.txt_'. We will be using the visible part of the testing data to understand the working on Apache Spark.  The user's listening history is provided as: (user, song, play count). In another file, '_kaggle_songs.txt_', each song is marked using an index for easier representation of songs. 



What to turn in:
A zip folder which will have:
- Jupyter Notebook
- A brief report on what features you used for recommendation. And a brief explanation of flow of your code. For example,  what RDD does what or, why it was created.
- datasets folder with the csv files you are using in your notebook.
- Notebook should use relative path to the csv files in datasets folder.
- Name of the zip folder - <your\_name>\_<your\_partner_name>.zip


This project consists of 4 questions. 
1. Create an RDD with _msd_evalutation_triplets.txt_ and replace the song name with the song index from _msd_songs.txt_. Identify the number of songs that do not have any rating. 
2. Generate song ratings based on the song play count as a normalized score between 0 and 1. 
3. Identify the popular song based on this rating and recommend songs to user, given user id based on the algorithm used in Movie recommender system from class. 
4. Using Cosine similarity function, identify pair-wise similarity between each pair of users and generate the top 5 most similar users without an overlap in users. 

The above list is the higer level idea about the questions. 

In [1]:
### Starter code ####

##### These lines are to tell jupyter where to find Apache Spark ####
import findspark
findspark.init('C:\\apachespark')
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local[*]").setAppName("Songs")
# There are two configurable parameters
# 1. A cluster URL, namely  local  in this example, which tells Spark how to connect
# to a cluster.  local  is a special value that runs Spark on one thread on the local
# machine, without connecting to a cluster.
# 2. An application name, namely  Movies  in this example. This will identify your
# application on the cluster manager’s UI if you connect to a cluster.
sc = SparkContext(conf = conf)
##### These lines are to tell jupyter where to find Apache Spark ####

In [2]:
## Read triplet file into RDD
triplet_rdd = sc.textFile(r"kaggle_visible_evaluation_triplets.txt") \
    .map(lambda line: line.split("\t")) 

triplet_rdd.collect()[0]

['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', 'SOBONKR12A58A7A7E0', '1']

In [3]:
## Read song file into RDD
songs_rdd = sc.textFile(r"kaggle_songs.txt") \
    .map(lambda line: line.split(" ")) 

songs_rdd.take(1)

[['SOAAADD12AB018A9DD', '1']]

## Step 1: 
Replace song name with song index and identify the number of songs without user history

In [4]:
## Bring the Song name as the first element, make the rest into a list
reordered_triplet_rdd = triplet_rdd.map(lambda x : (x[1], [x[0], x[2]]))
reordered_triplet_rdd.take(3)

[('SOBONKR12A58A7A7E0', ['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', '1']),
 ('SOEGIYH12A6D4FC0E3', ['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', '1']),
 ('SOFLJQZ12A6D4FADA6', ['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', '1'])]

In [5]:
## Convert songs_rdd into a list of tuples
songs_tuple_rdd = songs_rdd.map(lambda x : (x[0], x[1]))
songs_tuple_rdd.take(1)

[('SOAAADD12AB018A9DD', '1')]

In [6]:
## Join songs_tuple_rdd & reordered_triplet_rdd -> make sure the left table is songs_tuple_rdd!!
joined_rdd = songs_tuple_rdd.leftOuterJoin(reordered_triplet_rdd)
joined_rdd.take(3)

[('SOAAALJ12AB01828B4', ('12', None)),
 ('SOAAAQN12AB01856D3',
  ('16', ['32cf63cf65787ce7e72fc7fda6ee585979af6582', '1'])),
 ('SOAAAQN12AB01856D3',
  ('16', ['d1b88940eabd4fab860edf68f0f0842e4f902c78', '1']))]

In [7]:
## Get rid of the first index x[0] (song name) and replace it with song index (id)
replaced_rdd = joined_rdd.map(lambda x : (x[1][0], x[1][1]))
replaced_rdd.take(2)

[('12', None), ('16', ['32cf63cf65787ce7e72fc7fda6ee585979af6582', '1'])]

In [8]:
## Filter for songs that do not have user rating
no_rating_rdd = replaced_rdd.filter(lambda x : x[1] is None )
print("There are %d songs that do not have any ratings." % (no_rating_rdd.count()))

There are 223007 songs that do not have any ratings.


## Step 2:
Generate song ratings based on the play_count. For example, if (song_1, 5; song_2, 10; song_3, 5) i.e., song_1 is played 5 times, song_2 is played 10 times and song_3 is played 5 times, the normalized rating score should be 0.25, 0.5 and 0.25 respectively. 
Similarly, generate the rating for all the songs. You may notice that based on all songs, the rating is almost always very low. So, think of the best way to convert song count to ratings. (Hint: Try generating ratings based on each user's song play history)

In [9]:
from operator import add

In [10]:
## Read user file into RDD
users_rdd = sc.textFile(r"kaggle_users.txt") \
    .map(lambda line: line.split("\n")) 

users_rdd.take(1)

[['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d']]

In [11]:
## Make user ID the key for triplet RDD
triplet_user_rdd = triplet_rdd.map(lambda x : (x[0], int(x[2])))
triplet_user_rdd.take(1)

[('fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', 1)]

In [12]:
## Reduce By Key to add the number of plays (x[2]), for each user
sum_rdd = triplet_user_rdd.reduceByKey(add)
sum_rdd.take(3)

[('d7083f5e1d50c264277d624340edaaf3dc16095b', 17),
 ('d68dc6fc25248234590d7668a11e3335534ae4b4', 6),
 ('fdf6afb5daefb42774617cf223475c6013969724', 10)]

In [26]:
## Get rid of rows that do not have a play count
no_none_rdd = replaced_rdd.filter(lambda x : x[1] is not None )
no_none_rdd.take(2)

[('16', ['32cf63cf65787ce7e72fc7fda6ee585979af6582', '1']),
 ('16', ['d1b88940eabd4fab860edf68f0f0842e4f902c78', '1'])]

In [28]:
## Reorder to make username the key
user_key_rdd = no_none_rdd.map(lambda x : (x[1][0], (x[0], x[1][1])))
user_key_rdd.take(2)

[('32cf63cf65787ce7e72fc7fda6ee585979af6582', ('16', '1')),
 ('d1b88940eabd4fab860edf68f0f0842e4f902c78', ('16', '1'))]

In [29]:
## Join sum_rdd with user_key_rdd 
joined_user_sum = sum_rdd.leftOuterJoin(user_key_rdd)
joined_user_sum.take(2)
## (username, (number of times that user played songs, (song ID, number of times that song was played)))

[('6530c4fc41b9110de5d39fe0355fa103c66385f0', (44, ('36681', '5'))),
 ('6530c4fc41b9110de5d39fe0355fa103c66385f0', (44, ('75497', '8')))]

In [33]:
## Divide # of times that song was played by total number of times a user played all songs
rating_rdd = joined_user_sum.map(lambda x : (x[0], x[1][1][0], int(x[1][1][1])/int(x[1][0])))
rating_rdd.take(2)

[('6530c4fc41b9110de5d39fe0355fa103c66385f0', '36681', 0.11363636363636363),
 ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '75497', 0.18181818181818182)]

In [89]:
## Normalize the ratings
import math
normalized_rdd = rating_rdd.map(lambda x : (x[0], x[1], 10+math.log(x[2])))
normalized_rdd.take(5)

[('6530c4fc41b9110de5d39fe0355fa103c66385f0', '36681', 7.825248278515839),
 ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '75497', 8.295251907761575),
 ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '245936', 7.3144226547498485),
 ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '383719', 6.2158103660817385),
 ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '129546', 8.161720515137052)]

## Step 3: 
For a given user_id, rating, recommend 5 other songs from the list. One way to do this is based on another user who liked the same song liked by this user with rating more than the given rating and recommend the 5 songs based on the matched user's rating. 

In [40]:
## Inputs
givenUserId = '6530c4fc41b9110de5d39fe0355fa103c66385f0';
givenSongId = '36681';
givenRating = 7.825248278515839;

In [49]:
## Reformat normalized_rdd
reformatted_rdd = normalized_rdd.map(lambda x : (x[0], [x[1], x[2]]))
reformatted_rdd.take(2)

[('6530c4fc41b9110de5d39fe0355fa103c66385f0', ['36681', 7.825248278515839]),
 ('6530c4fc41b9110de5d39fe0355fa103c66385f0', ['75497', 8.295251907761575])]

In [60]:
## Function to find users that liked the same song more than the given user
def liked(col):
    if ((col[0] != '') and (col[1][0] == givenSongId) and (float(col[1][1]) > givenRating)): #does the movie id match, rating higher?
        return True
    return False

## We just need the users that have liked that song more than given user
matched_users_rdd = reformatted_rdd.filter(liked)
matched_users_rdd.take(2)

## These are the users that we are focusing on - these users like the song more than given user

[('6385fa76b90e95f49909701b5effcdc17ea3bc0a', ['36681', 8.208240530771945]),
 ('af71c172969f8918b58b7d160177e5e332332bc5', ['36681', 8.3905620875659])]

In [61]:
## Get rid of song idex - same song
just_users_rdd = matched_users_rdd.map(lambda x : (x[0], x[1][1]) )
just_users_rdd.take(2)

[('6385fa76b90e95f49909701b5effcdc17ea3bc0a', 8.208240530771945),
 ('af71c172969f8918b58b7d160177e5e332332bc5', 8.3905620875659)]

In [62]:
## Find all the songs that the users have listened to by joining
## Inner join between reformatted_rdd and this user list - the user name needs to match with the just_users_rdd
## Get other songs that that user has listened to!
potential_songs = reformatted_rdd.join(just_users_rdd)
potential_songs.take(2)

[('6385fa76b90e95f49909701b5effcdc17ea3bc0a',
  (['36681', 8.208240530771945], 8.208240530771945)),
 ('6385fa76b90e95f49909701b5effcdc17ea3bc0a',
  (['288097', 8.208240530771945], 8.208240530771945))]

In [64]:
## Reformat the potential songs
reformatted_potential_rdd = potential_songs.map(lambda x : (x[1][0][0], [x[0], x[1][0][1]]))
reformatted_potential_rdd.take(2)

[('36681', ['6385fa76b90e95f49909701b5effcdc17ea3bc0a', 8.208240530771945]),
 ('288097', ['6385fa76b90e95f49909701b5effcdc17ea3bc0a', 8.208240530771945])]

In [65]:
## Function to exclude the same song as given song, because right now in our list there is also our given song
def notTheGivenSong(row):
    if(row[0] != givenSongId):
        return True
    return False

songs_without_given = reformatted_potential_rdd.filter(notTheGivenSong)
songs_without_given.take(2)

[('288097', ['6385fa76b90e95f49909701b5effcdc17ea3bc0a', 8.208240530771945]),
 ('385781', ['6385fa76b90e95f49909701b5effcdc17ea3bc0a', 8.208240530771945])]

In [75]:
## Drop the users from RDD
songs_without_users = songs_without_given.map(lambda x : (x[0], float(x[1][1])))
songs_without_users.takeOrdered(5, key=lambda x: -x[1])

[('217663', 9.393864196429684),
 ('248603', 9.21154263963573),
 ('240580', 9.18906978378367),
 ('343755', 9.152702139612796),
 ('302439', 9.083709268125846)]

In [74]:
## Aggregate the ratings for each song, and take the top 5 songs with the highest rating
songs_aggregated = songs_without_users.reduceByKey(add) 
songs_aggregated.takeOrdered(5, key=lambda x: -x[1])

[('328524', 23.75002475774052),
 ('215861', 15.030186700423998),
 ('286747', 14.96304739758637),
 ('36622', 14.36521039683075),
 ('242113', 13.750990474860476)]

In [82]:
## Find the song names that match the top 5 song indexes
## Reformat our songs_tuple to have the song index as key
songs_tuple_reordered = songs_tuple_rdd.map(lambda x : (x[1], x[0]))
songs_tuple_reordered.take(2)

[('1', 'SOAAADD12AB018A9DD'), ('2', 'SOAAADE12A6D4F80CC')]

In [84]:
## Join the song_tuple with our top 5 songs rdd
song_names = songs_aggregated.join(songs_tuple_reordered)
song_names.take(2)

[('281084', (8.3905620875659, 'SOSJRXV12A8C136E1B')),
 ('123485', (7.227411277760218, 'SOHTEDD12A6D4F8215'))]

In [85]:
## Reformat song_names to drop song index, and include just the song rating
song_names_reformatted = song_names.map(lambda x : (x[1][1], x[1][0]))
song_names_reformatted.take(2)

[('SOSJRXV12A8C136E1B', 8.3905620875659),
 ('SOHTEDD12A6D4F8215', 7.227411277760218)]

In [87]:
## Take the top five from the song names list
song_names_reformatted.takeOrdered(5, key=lambda x : -x[1])

[('SOVTLQW12AB0186641', 23.75002475774052),
 ('SONVPTP12A6D4F7A34', 15.030186700423998),
 ('SOSUADS12A58A80A47', 14.96304739758637),
 ('SOCGXXL12B0B808865', 14.36521039683075),
 ('SOPQJAZ12A6310F168', 13.750990474860476)]

## Step 4: 
1. Compute cosine similarity between all pairs of users. 
2. Sort the similarity score and print the top-5 similar users. 
3. If the top-5 user set has an user appearing more than once, ignore that pair and take the next best pair from the sorted list. 
4. For a given user_id, identify the top-5 similar users and hence song recommendations from other user's list. 